<a href="https://colab.research.google.com/github/shoreey/Vehicle-REID/blob/main/Vehicle_Reidentification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Note: This code snippet is provided for educational and academic purposes.
# The actual implementation was performed in a CLI conda environment and PyCharm.
# Please refer to the documentation to get familiar with the environments and ensure you have the required libraries installed.


**Single Camera Vehicle tracking**

In [ ]:
import cv2
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

# Open the video file
video_path = "E:\\Data Science\\Dissertation\\Shaurya\\AIC22_Track1_MTMC_Tracking\\DATASET\\c041.mp4"
cap = cv2.VideoCapture(video_path)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()
"""

**image Cropping** **This code should be executed in the CLI**

In [ ]:
yolo task=detect mode=predict model=yolov8n.pt source=c041.mp4 tracker=botsort.yaml save_crop=True hide_labels=True hide_conf=True

In [ ]:
Feature Extractor #using example of HOG for getting started, RESNET-50 was used in the actual implementation



```
# This is formatted as code
```



In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

# Set paths to the directories containing cropped truck objects
camera1_truck_dir = 'C:/Users/shaur/runs/detect/predict/crops/truck'
camera2_truck_dir = 'C:/Users/shaur/runs/detect/predict2/crops/truck'

# Initialize a feature extraction model (you can use a pretrained model)
# For example, using OpenCV's HOG (Histogram of Oriented Gradients) for demonstration:
hog = cv2.HOGDescriptor()

# Initialize lists to store extracted features and their corresponding cameras
truck_features = []
cameras = []

# Function to extract features from a list of images
def extract_features_from_images(image_paths, feature_extractor):
    features = []
    for image_path in image_paths:
        image = cv2.imread(image_path)
        if image is not None:
            feature = feature_extractor.compute(image)
            features.append(feature)
    return features

# Extract features for Camera 1 (c041)
camera1_image_paths = [os.path.join(camera1_truck_dir, filename) for filename in os.listdir(camera1_truck_dir)]
camera1_features = extract_features_from_images(camera1_image_paths, hog)
truck_features.extend(camera1_features)
cameras.extend(['c041'] * len(camera1_features))

# Extract features for Camera 2 (c042)
camera2_image_paths = [os.path.join(camera2_truck_dir, filename) for filename in os.listdir(camera2_truck_dir)]
camera2_features = extract_features_from_images(camera2_image_paths, hog)
truck_features.extend(camera2_features)
cameras.extend(['c042'] * len(camera2_features))

# Convert features and camera labels to NumPy arrays
truck_features = np.array(truck_features)
cameras = np.array(cameras)

# Create a Pandas DataFrame to store the features and camera labels
data = {'Feature': list(truck_features), 'Camera': cameras}
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('truck_features.csv', index=False)

**Query System #feedback Mechanism**

In [ ]:
import cv2
import pandas as pd
import os

# Load the Excel file containing truck filenames
matched_trucks_df = pd.read_excel("C:/Users/shaur/PycharmProjects/pythonProject3/matching_trucks.xlsx")

# Directory paths for c041 and c042
c041_dir = 'C:/Users/shaur/runs/detect/predict/crops/truck'
c042_dir = 'C:/Users/shaur/runs/detect/predict2/crops/truck'

# Function to display matched trucks and gather user feedback
def display_matched_trucks(truck1_filename, truck2_filename):
    truck1_image_path = os.path.join(c041_dir, truck1_filename)
    truck2_image_path = os.path.join(c042_dir, truck2_filename)

    # Load images with error handling
    truck1_image = cv2.imread(truck1_image_path)
    truck2_image = cv2.imread(truck2_image_path)

    if truck1_image is None or truck2_image is None:
        print(f"Error loading one or both images: {truck1_image_path}, {truck2_image_path}")
        return

    # Display images
    cv2.imshow('Matched Truck from Camera 1', truck1_image)
    cv2.imshow('Matched Truck from Camera 2', truck2_image)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Function to display matched trucks and gather user feedback
def display_and_confirm_matches(truck1_filename, truck2_filename):
    display_matched_trucks(truck1_filename, truck2_filename)

    # Gather user feedback
    feedback = input("Are these matches correct? (y/n): ")

    return feedback.lower() == 'y'

# Create a list to store user feedback and matching column
user_feedback = []
matching_column = []  # New column to store 'Yes' or 'No'

# Iterate through the matched truck pairs and gather user feedback
for idx, row in matched_trucks_df.iterrows():
    truck1_filename = row['Truck1']
    truck2_filename = row['Truck2']

    print(f"Pair: {truck1_filename}, {truck2_filename}")

    if os.path.exists(os.path.join(c041_dir, truck1_filename)) and os.path.exists(
            os.path.join(c042_dir, truck2_filename)):
        if display_and_confirm_matches(truck1_filename, truck2_filename):
            user_feedback.append('Yes')
            matching_column.append('Yes')
        else:
            user_feedback.append('No')
            matching_column.append('No')
    else:
        print(f"Skipping missing images: {truck1_filename}, {truck2_filename}")
        user_feedback.append('N/A')
        matching_column.append('N/A')

# Add user feedback and matching column to the DataFrame
matched_trucks_df['UserFeedback'] = user_feedback
matched_trucks_df['Matching'] = matching_column

# Save the results to an Excel file
matched_trucks_df.to_excel('matched_trucks_with_feedback.xlsx', index=False)


**Assigning Id's # Vehicle Reidentification**

In [ ]:
import pandas as pd
import cv2
import tkinter as tk
import numpy as np  # Add this import

# Load the Excel file containing matched trucks with feedback and assigned IDs
matched_trucks_df = pd.read_excel('C:/Users/shaur/PycharmProjects/pythonProject3/matched_trucks_with_feedback.xlsx')

# Directory paths for c041 and c042
c041_dir = 'C:/Users/shaur/runs/detect/predict/crops/truck'
c042_dir = 'C:/Users/shaur/runs/detect/predict2/crops/truck'

# Function to display matched trucks and their IDs
def display_matched_trucks_with_ids(truck1_filename, truck2_filename, assigned_id):
    truck1_image_path = os.path.join(c041_dir, truck1_filename)
    truck2_image_path = os.path.join(c042_dir, truck2_filename)

    # Load images with error handling
    truck1_image = cv2.imread(truck1_image_path)
    truck2_image = cv2.imread(truck2_image_path)

    if truck1_image is None or truck2_image is None:
        print(f"Error loading one or both images: {truck1_image_path}, {truck2_image_path}")
        return

    # Display images and IDs for non-NA values
    if not pd.isna(assigned_id):
        cv2.putText(truck1_image, f"ID: {assigned_id}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(truck2_image, f"ID: {assigned_id}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Show images with IDs
        cv2.imshow('Matched Truck from Camera 1', truck1_image)
        cv2.imshow('Matched Truck from Camera 2', truck2_image)

        cv2.waitKey(0)
        cv2.destroyAllWindows()

# Create a Tkinter window
root = tk.Tk()
root.withdraw()  # Hide the main window

# Iterate through the rows of the DataFrame
for idx, row in matched_trucks_df.iterrows():
    truck1_filename = row['Truck1']
    truck2_filename = row['Truck2']
    assigned_id = row['AssignedID']

    # Display matched trucks with their assigned IDs (skip rows with 'NA' in AssignedID)
    if not pd.isna(assigned_id):
        display_matched_trucks_with_ids(truck1_filename, truck2_filename, assigned_id)

# Close the Tkinter window
root.destroy()


**Traffic Tracking and Visualization**

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from itertools import cycle
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

# Open the video file
video_path = "path/to/video.mp4"
cap = cv2.VideoCapture(video_path)

# Store the track history
track_history = defaultdict(lambda: [])

# Lists to store the final positions for plotting
all_x_positions = []
all_y_positions = []

# Define a cycle of colors for tracks
track_colors = cycle(['b', 'g', 'r', 'c', 'm', 'y', 'k'])

# Maximum number of tracks to display
max_tracks = 9

# Maximum number of labels to display
max_labels = 7

# Lists to store the legend labels and colors
legend_labels = []
legend_colors = []

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True)

        # Get the boxes and track IDs
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        # Plot the tracks and labels
        plotted_labels = set()  # To keep track of plotted labels
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))  # x, y center point
            if len(track) > max_tracks:  # limit the number of points in the track history
                track.pop(0)

            # Append the positions to the lists
            all_x_positions.extend([point[0] for point in track])
            all_y_positions.extend([point[1] for point in track])

            # Get the next unique color for the current track
            color = next(track_colors)

            # Plot the track with label (up to max_labels)
            if len(plotted_labels) < max_labels:
                label = f"Track {track_id}"
                plt.plot([point[0] for point in track], [point[1] for point in track], label=label, color=color)
                legend_labels.append(label)
                legend_colors.append(color)
                plotted_labels.add(label)

    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object
cap.release()

# Create the final plot without a legend
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.title('Traffic Movement')
plt.grid(True)

# Move the legend outside of the graph
plt.legend(handles=[plt.Line2D([0], [0], color=color, label=label) for color, label in zip(legend_colors[:max_labels], legend_labels[:max_labels])],
           loc='center left', bbox_to_anchor=(1, 0.5))
plt.subplots_adjust(right=0.7)  # Adjust the space for the legend
plt.show()